In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import statsmodels.api as sm
%matplotlib inline

In [2]:
df_grades = pd.read_csv('data/Grades-Table 1.csv')#
df_demographics = pd.read_csv('data/Demographics-Table 1.csv')#
df_student_aff1 = pd.read_csv('data/Student Affective 1-Table 1.csv')#
df_student_aff2 = pd.read_csv('data/Student Affective 2-Table 1.csv')#
df_study1 = pd.read_csv('data/Study Habits 1-Table 1.csv')#
df_study2 = pd.read_csv('data/Study Habits 2-Table 1.csv')#

# Feature engineering, some averageing, dummy variables, drop columns, etc.

In [3]:
# drop some columns from grades that I don't need
df_grades = df_grades.drop('Scientists_ID', axis=1)
df_grades = df_grades.drop('TOTAL_POINTS', axis=1)
df_grades = df_grades.drop('Letter_Grade_Given', axis=1)

# make some categorical variables columns
df_demographics = pd.get_dummies(df_demographics, columns = ['STEM_Major', 'Ethnicity', 'sexual', 'edu_lvl'])

# make some categorical variable columns
df_grades = pd.get_dummies(df_grades, columns = ['Section','Time','Treatment'])

## Linear Regression with Grades, Demographics, Student Affective 2, and Study Habits 2

In [4]:
# merging all of the tables together to make one super table
df_merged = df_grades.merge(df_demographics, left_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', \
                                                right_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', how = 'outer')
# df_merged = df_merged.merge(df_student_aff1, left_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', \
#                                                 right_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', how = 'outer')
df_merged = df_merged.merge(df_student_aff2, left_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', \
                                                right_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', how = 'outer')
# df_merged = df_merged.merge(df_study1, left_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', \
#                                                 right_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', how = 'outer')
df_merged = df_merged.merge(df_study2, left_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', \
                                                right_on = 'SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', how = 'outer')
df_merged = df_merged.dropna() # all tables merged and rows with missing data have been removed
df_merged.reset_index(drop=True, inplace=True) # just resets the indexes after all that dropping
df_merged = df_merged.drop('SECTION_INSTRUCTOR_TIME_TREATMENT_STUDENT', axis=1)
df_merged.head()

,Instructor,Student,Microscope_Quiz,Microscopes_HW,Sci_Method_Quiz,Daphnia_HW,Transport_Quiz,Transport_HW,Daphnia_PR,Metabolism_Quiz,...,Co_I_understand,Read_B4,Bk_Notes,Re_read,Resources,Q_Notes,Peer_?s,Inst_?s,Peer_Comm,connections
0,7.0,1.0,0.0,30.0,20.0,60.0,26.0,20.0,50.0,29.0,...,1.0,5.0,4.0,2.0,3.0,3.0,2.0,3.0,2.0,3.0
1,7.0,2.0,16.0,30.0,20.0,25.0,28.0,28.0,50.0,21.0,...,3.0,4.0,4.0,1.0,4.0,1.0,5.0,5.0,2.0,1.0
2,7.0,3.0,24.0,21.0,23.0,57.0,18.0,28.0,50.0,29.0,...,2.0,3.0,5.0,2.0,3.0,1.0,2.0,3.0,1.0,1.0
3,7.0,4.0,22.0,28.0,18.0,60.0,19.0,30.0,50.0,9.0,...,4.0,4.0,4.0,1.0,1.0,2.0,4.0,3.0,4.0,3.0
4,7.0,5.0,20.0,28.0,15.5,53.0,26.0,30.0,50.0,15.0,...,4.0,4.0,5.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0


In [5]:
X = df_merged.drop('FINAL_RAW_BASE_GRADE', axis=1).astype(float)
X['const'] = 1
y = df_merged.FINAL_RAW_BASE_GRADE

In [6]:
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     FINAL_RAW_BASE_GRADE   R-squared:                       1.000
Model:                              OLS   Adj. R-squared:                  1.000
Method:                   Least Squares   F-statistic:                 2.018e+04
Date:                  Sat, 09 Jul 2016   Prob (F-statistic):          1.62e-110
Time:                          22:17:04   Log-Likelihood:                 816.96
No. Observations:                   140   AIC:                            -1470.
Df Residuals:                        58   BIC:                            -1229.
Df Model:                            81                                         
Covariance Type:              nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Instructor                          -0.0017      0.000     -6.138      0.000      -0.002      -0.001
Student                          -2.102e-05   2.75e-05     -0.763      0.448   -7.62e-05    3.41e-05
Microscope_Quiz                  -5.192e-06   6.69e-05     -0.078      0.938      -0.000       0.000
Microscopes_HW                   -8.079e-06   7.14e-05     -0.113      0.910      -0.000       0.000
Sci_Method_Quiz                  -3.971e-05   6.78e-05     -0.586      0.560      -0.000     9.6e-05
Daphnia_HW                        -9.22e-05   7.49e-05     -1.231      0.223      -0.000    5.78e-05
Transport_Quiz                   -4.214e-05   8.32e-05     -0.506      0.615      -0.000       0.000
Transport_HW                     -4.387e-05   7.38e-05     -0.595      0.554      -0.000       0.000
Daphnia_PR                        3.879e-05    7.5e-05      0.517      0.607      -0.000       0.000
Metabolism_Quiz                  -2.164e-05   7.87e-05     -0.275      0.784      -0.000       0.000
Metabolism_HW                    -2.494e-05   7.13e-05     -0.350      0.728      -0.000       0.000
Mitosis_Quiz                     -5.886e-06   7.41e-05     -0.079      0.937      -0.000       0.000
Mitosis_HW                       -2.411e-05   7.17e-05     -0.336      0.738      -0.000       0.000
Meiosis_Quiz                     -6.124e-05   8.24e-05     -0.743      0.461      -0.000       0.000
Meisosis_HW                      -7.491e-05   7.26e-05     -1.032      0.307      -0.000    7.04e-05
Phylogenetic_Quiz                -7.417e-05   7.61e-05     -0.975      0.334      -0.000    7.81e-05
Phylogenetic_HW                  -1.743e-05   6.78e-05     -0.257      0.798      -0.000       0.000
Evolution_Quiz                   -4.131e-05   7.55e-05     -0.547      0.586      -0.000       0.000
Evolution_HW                     -1.563e-05   7.13e-05     -0.219      0.827      -0.000       0.000
Plant_Quiz                       -7.971e-05   8.45e-05     -0.944      0.349      -0.000    8.94e-05
Plant_HW                         -7.672e-05   8.35e-05     -0.919      0.362      -0.000    9.04e-05
Animal_Quiz                       3.387e-05    7.3e-05      0.464      0.645      -0.000       0.000
Animal_HW                        -3.183e-05   8.29e-05     -0.384      0.702      -0.000       0.000
Pop_Quiz                          -8.08e-06   8.04e-05     -0.100      0.920      -0.000       0.000
Pop_HW                           -5.147e-06   6.48e-05     -0.079      0.937      -0.000       0.000
Ecology_Quiz                     -6.077e-05   7.68e-05     -0.791      0.432      -0.000     9.3e-05
Ecology_HW                       -3.452e-05   7.45e-05     -0.464      0.645      -0.000       0.000
Co-Instruction                 